<a href="https://colab.research.google.com/github/BJHeemskerk/DeepLearning_2024/blob/Henry/Opdracht%201/Portfolio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Voorspellen van huisprijzen**

|Teamleden|Kaggle Username|GitHub Username|
|--|--|--|
|Nima Ghafar|NimaGhafar|NimaGhafar|
|Busse Heemskerk|bussejheemskerk|BJHeemskerk|
|Henry Lau|thugwithin|HenryLau08|
|Jesse van Leeuwen|22096337|22096337|

-- Opdrachtomschrijving --

## Inhoudsopgave
1. [H1: Importeren en inladen](#h1)
2. [H2: EDA](#h2)
3. [H3: Fully Connected Neural Network](#h3)

## H1: Importeren en inladen <a name='h1'></a>
Eerst zullen de verschillende gebruikte libaries worden geimporteerd.

In [1]:
# Importeren van veel gebruikte libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
import datetime
# Importeren libaries voor aanpassen/behandelen features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Importeren DL libaries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping

Nu zal de data worden ingeladen. Om hiermee te beginnen zullen de csv bestanden worden ingeladen.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/DeepLearning_2024/Opdracht 1')


In [ ]:
# Inladen van csv bestanden
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Tonen van de data
display(train_data.head(), test_data.head())

In de volgende code cel wordt de data van elke afbeelding in de train en de test map opgehaald.

In [ ]:
# Aanmaken van te vullen lijsten
file_paths_tr = []
imgdata_tr = []
file_paths_te = []
imgdata_te = []

# Opzetten directories
directories = ["Train", "Test"]

# Ophalen van de bestanden
# Loopen over directories
for directory in directories:
    directory_path = os.path.join('/content/drive/My Drive/Colab Notebooks/DeepLearning_2024/Opdracht 1', directory)
    # Loopen over alle bestanden
    for file in os.listdir(directory_path):
        # Alleen checken van .jpg bestanden
        if file.endswith(".jpg"):
            # Aanmaken van pad naar het bestand.
            file_path = os.path.join(directory_path, file)

            # Openen van de afbeelding en omzetten naar np.array
            img = Image.open(file_path)
            img_array = np.array(img)

            # Vullen van juiste lijsten op basis van ID
            if int(file.split('.')[0]) > 500:  # We halen de ID uit de bestandsnaam
                file_paths_te.append(file_path)
                imgdata_te.append(img_array)
            else:
                file_paths_tr.append(file_path)
                imgdata_tr.append(img_array)


In [ ]:
# Aanmaken van te vullen lijsten
file_paths_tr = []
imgdata_tr = []
file_paths_te = []
imgdata_te = []

# Opzetten directories
directories = ["Train/", "Test/"]

# Ophalen van de bestanden
# Loopen over directories
for directory in directories:
    # Loopen over alle bestanden
    for file in os.listdir(directory):
        # Alleen checken van .jpg bestanden
        if file.endswith(".jpg"):
            # Aanmaken van pad naar het bestand.
            file_path = os.path.join(directory, file)

            # Openen van de afbeelding en omzetten naar np.array
            img = Image.open(file_path)
            img_array = np.array(img)

            # Ophalen numerieke ID van het beastand
            file_number = ''.join(filter(str.isdigit, file))

            # Vullen van juiste lijsten op basis van ID
            if file_number and int(file_number) > 500:
                file_paths_te.append(file_path)
                imgdata_te.append(img_array)
            else:
                file_paths_tr.append(file_path)
                imgdata_tr.append(img_array)

Nu alle data in lijsten is gezet kunnen er dataframes worden aangemaakt met de data van de afbeeldingen. Door de grootte van de arrays kan dit even duren.

In [ ]:
# Initieren van het dataframe
train_images = pd.DataFrame(columns=["House ID", "FilePath", "ImgData"])
test_images = pd.DataFrame(columns=["House ID", "FilePath", "ImgData"])

# Aanvullen train dataset
train_images["House ID"] = train_data["House ID"]
train_images["FilePath"] = file_paths_tr
train_images["ImgData"] = imgdata_tr

# Aanvullen test dataset
test_images["House ID"] = test_data["House ID"]
test_images["FilePath"] = file_paths_te
test_images["ImgData"] = imgdata_te

# Tonen van de dataframes
display(train_images.head(), test_images.head())

Vervolgens zullen de dataframes worden getest door de afbeeldingen te tonen.

In [ ]:
# Ophalen van pad naar afbeelding
image_path = train_images["FilePath"][0]

# Openen afbeelding
img = Image.open(image_path)

# Omzetten afbeelding naar np.array
img_array = np.array(img)

# Tonen van de afbeelding
plt.imshow(img)
plt.show()

# Opnieuw tonen van de afbeelding via DF
plt.imshow(train_images["ImgData"][0])
plt.show()

## H2: EDA <a name='h2'></a>

- [ ] Voer een Exploratieve Data Analyse (EDA) uit:
    - [ ] Lees de afbeeldingen in en toon enkele afbeeldingen.
    - [ ] Laad het metadatabestand en maak relevante visualisaties om de relaties tussen de targetvariabele en de andere variabelen te onderzoeken.
- [ ] Beschrijf wat de belangrijkste bevindingen zijn van de EDA.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns

# Load the metadata
train_df = pd.read_csv(r'train.csv')
test_df = pd.read_csv(r'test.csv')

# Display the first few rows of the dataframe
display(train_df.head())
display(test_df.head())

# Read and display images
train_images_dir = r'Train'
test_images_dir = r'Test'

# Display some images from the train set
train_images = os.listdir(train_images_dir)
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = cv2.imread(os.path.join(train_images_dir, train_images[i]))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')

# Display some images from the test set
if 'Price' not in train_df.columns:
    raise KeyError("Column 'target' does not exist in the dataframe.")


In [ ]:
# EDA on train dataframe
print("Train dataframe shape:", train_df.shape)

# Use describe() for summary statistics
print("Train dataframe summary statistics:")
display(train_df.describe(include='all'))

# Use info() for data types and missing values
print("Train dataframe info:")
display(train_df.info())

# EDA on test dataframe
print("Test dataframe shape:", test_df.shape)

# Use describe() for summary statistics
print("Test dataframe summary statistics:")
display(test_df.describe(include='all'))

# Use info() for data types and missing values
print("Test dataframe info:")
display(test_df.info())

# Visualize the distribution of the target variable in the train dataframe
plt.figure(figsize=(10, 6))
sns.histplot(train_df['Price'], kde=True)
plt.title('Distribution of Price in Train Data')
plt.xlabel('Price')
plt.ylabel('Count')
plt.show()

sns.pairplot(train_df)
plt.show()



In [ ]:
# Maak een heatmap met  cmap='RdYlGn' alleen op "Price" in de train dataframe en in volgorde van hoog naar laag
plt.figure(figsize=(10, 6))
sns.heatmap(train_df.corr()[['Price']].sort_values(by='Price', ascending=False), annot=True, cmap='RdYlGn')
plt.title('Correlation of Features with Price')
plt.show()

# Maak een heatmap met  cmap='RdYlGn' voor alle kolommen in de train dataframe en in volgorde van hoog naar laag

plt.figure(figsize=(10, 6))
sns.heatmap(train_df.corr(), annot=True, cmap='RdYlGn')
plt.title('Correlation of Features')
plt.show()


## H3: Splitsen en scalen

Voordat er neurale netwerken kunnen worden opgesteld, zal eerst nog de train_data dataset worden opgesplitst in een test en een validatie set.

In [ ]:
# Aanmaken van de random_state SEED
SEED = 42

# Opslitsten van data in X en y
X = train_data.drop(['House ID', 'Price'], axis=1)
y = train_data['Price']

# Toepassen van train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                        X, y, test_size=0.1, random_state=SEED
                                       )

Vervolgens moet de data worden gescaled, dit zorgt ervoor dat het model betere verbanden kan leggen tussen de verschillende variabelen.

In [ ]:
# Scalen van de gesplitste data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Scalen van de test_data
# Aanmaken lijst met nodige kolomnamen
kolomnamen = ["Bedrooms", "Bathrooms", "Area", "Latitude", "Longitude"]

# Scalen van de nodige kolommen
test_fe = test_data[kolomnamen]
sc_test_fe = scaler.fit_transform(test_fe)

**versie 1 van het model**

## H3: Fully Connencted Neural Network <a name='h3'></a>

Nu kan er worden begonnen met het opzetten van verschillende voorspellingen. Het eerste type voorspelling dat voor zal komen is een Fully Connected Neural Network. Dit netwerk bestaat uit de volgende lagen:

**Input laag:**<br>
De input laag van een neuraal netwerk bestaat uit een bepaald aantal neuronen. Het aantal neuronen moet gelijk zijn aan het aantal features dat in de dataset beschikbaar is. In onze datasets zijn er 5 features te vinden, dus heeft de input laag 5 neuronen.

In alle hidden layers is gebruik gemaakt van een trial & error aanpak, waarmee er met verschillende hoeveelheden van neuronen en verschillende activatie functies is getest. Uiteindelijk kwamen hier de volgende aantallen en activaties uit:

**Hidden Layer 1:**<br>
- Neuronen: 128
- Activatie: Rectified Linear Unit

**Hidden Layer 2:**<br>
- Neuronen: 64
- Activatie: Rectified Linear Unit

**Hidden Layer 3:**<br>
- Neuronen: 64
- Activatie: Rectified Linear Unit

**Hidden Layer 4:**<br>
- Neuronen: 16
- Activatie: Rectified Linear Unit

Voor de output laag geldt ongeveer hetzelfde als voor de input laag. Het aantal neuronen in de output laag hoort gelijk te zijn aan het aantal klassen die worden voorspeld bij classificatie. Bij regressie taken is er vaak maar 1 neuron in de output laag aanwezig, omdat er een eenvoudige numerieke voorspelling wordt gemaakt. In ons regressie probleem is deze waarde de huisprijs. Bij regressie is er ook sprake van de activatie functie 'linear', ookwel bekend als geen activatie.

Nu de data is gesplitst zal er een Fully Connected Neural Network worden opgebouwd.

In [ ]:
# Aanmaken NN mbv Sequential API
fcnn = Sequential()

# Toevoegen input en eerste hidden layer
fcnn.add(Dense(128, input_shape=(5,), activation='relu'))

# Toevoegen andere hidden layers
fcnn.add(Dense(64, activation='relu'))
fcnn.add(Dense(64, activation='relu'))
fcnn.add(Dense(16, activation='relu'))

# Toevoegen van output layer
fcnn.add(Dense(1, activation='linear'))

Nadat alle lagen van het model zijn opgesteld, is het tijd om het model te compilen. Deze stap voegt enkele parameters toe aan het model:
- De optimizer, een algoritme dat bepaalt op welke wijze het model leert tijdens het trainen.
- De loss functie, een functie die aantoont hoe goed het model presteert op de gegevens.
- De metric, een extra, optionele meetwaarde voor de prestatie van het model.

Voor ons model is er gekozen voor de volgende parameters:
- De Adam (Adaptive Moment estimator) optimizer
- Mean Squared Error als loss
- Mean Absolute Percentage Error als metric.

De keuze voor de optimizer en de loss is gemaakt door een paar van de bekendste en meest gebruikte opties te nemen. Als metric is er gekozen voor de MAPE, omdat deze metric ook wordt gebruikt in de kaggle competitie.

Daarnaast is er ook een module aangemaakt om het model eerder te kunnen stoppen, in het geval dat er niet veel meer veranderd aan de loss van de validatie data.

In [ ]:
# Compilen van het model
fcnn.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanAbsolutePercentageError()]
)

# Maken van een vroege stop module
early_stop = EarlyStopping(
    monitor='val_loss', patience=25, restore_best_weights=True
    )

Nu alles is aangemaakt kan het model worden getrained. Hierbij gebruiken we de trainings data, de validatie data, een gekozen aantal epochs, een batch size en een callback module. De epochs staan voor het aantal iteraties dat het model door zal gaan, door gebruik van de callback module is er gekozen om 5000 epochs uit te voeren. Zodra de loss op de validatie set voor 100 epochs niet genoeg veranderd zal het model automatisch stoppen en worden de parameters van de beste epoch hersteld. De batch size wijst op het aantal datapunten dat per epoch wordt gebruikt, bij ons is er gekozen voor een batch size van 16.

In [ ]:
# Fitten van het model met de data
h_fcnn = fcnn.fit(
    X_train, y_train,
    batch_size=16,
    epochs=5000,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]
)

Met het getrainde model kan er gekeken worden naar de loss van de train data en de validatie data. Op deze manier kan er in een enkele grafiek worden gezien of er sprake is van overfitting of andere problemen.

In [ ]:
plt.plot(h_fcnn.history['loss'], label='Training loss')
plt.plot(h_fcnn.history['val_loss'], 'g', label='Validation loss')
plt.xlabel('Epoch #')
plt.ylabel('Loss (MAPE)')
plt.title('Loss grafiek voor FCNN-model')
plt.legend()
plt.show()

Zoals in de grafiek is te zien, is er sprake van geen van overfitting. Overfitting wordt namelijk aangeduid door het stijgen van de validatie loss.

Nu het model is getrained en de loss is bekeken, kan de voorspelling worden geupload naar kaggle. Aan de hand van de voorspellingen kan er een csv bestand aangemaakt worden voor de kaggle competitie. Omdat deze bestanden vaker moeten worden aangemaakt en om de bestanden overzichtelijk en uniform te houden is hiervoor een functie aangemaakt.

In [ ]:
model_save_path = '/content/drive/My Drive/Colab Notebooks/DeepLearning_2024/Opdracht 1/modelversies/fcnn_model_v1.h5'
fcnn.save(model_save_path)


In [ ]:
from tensorflow.keras.models import load_model


model_load_path = '/content/drive/My Drive/Colab Notebooks/DeepLearning_2024/Opdracht 1/modelversies/fcnn_model_v1.h5'


model = load_model(model_load_path)
model.summary()

**versie 2 van het model**


In [ ]:

# hier definieren we het modle
model = Sequential()

# Input layer en eerste hidden layer met Dropout
model.add(Dense(128, input_shape=(5,), activation='relu'))
model.add(Dropout(0.2))

# Tweede hidden layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# Derde hidden layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(1, activation='linear'))

# Compileer het model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train het model
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=1000,
                    validation_split=0.2,
                    callbacks=[early_stop])


## Modelaanpassingen en Verbeteringen

In deze versie van het neurale netwerkmodel zijn verschillende aanpassingen gemaakt ten opzichte van de eerste versie van het model. Deze wijzigingen zijn bedoeld om de modelprestaties te verbeteren, zoals gemeten door een lagere Mean Absolute Percentage Error (MAPE) op Kaggle. Hier volgt een overzicht van de belangrijkste aanpassingen:

### Architectuur van het Model
- **Verhoging van het Aantal Neuronen**: In elke laag van het model is het aantal neuronen aangepast om de modelcomplexiteit te verhogen. Dit stelt het model in staat om complexere patronen in de data te leren.
    - De inputlaag en de eerste verborgen laag bevatten nu 128 neuronen.
    - De tweede verborgen laag bevat 64 neuronen.
    - De derde verborgen laag bevat 32 neuronen.
- **Dropout Toegevoegd**: In elke verborgen laag is een dropout van 0.2 toegepast. Dropout helpt overfitting te verminderen door willekeurig een deel van de neuronen tijdens de training uit te schakelen, wat zorgt voor een robuuster netwerk.

### Hyperparameters
- **Learning Rate**: De learning rate van de Adam optimizer is ingesteld op 0.001. Dit is een standaardwaarde die vaak een goed evenwicht biedt tussen snelheid en nauwkeurigheid van de training.
- **Batch Size**: De batch size is vastgesteld op 32. Dit is een veelgebruikte grootte die een goede balans biedt tussen de snelheid van de training en de nauwkeurigheid van de gradient updates.
- **Aantal Epochs**: Er is gekozen voor een maximum van 1000 epochs, met early stopping toegepast om te voorkomen dat het model te lang traint en overfit raakt.

### Training en Validatie
- **Validation Split**: Een validatiesplit van 0.2 wordt gebruikt om een deel van de trainingsdata apart te zetten voor validatiedoeleinden. Dit helpt bij het monitoren van het modelprestaties op niet-getrainde data.
- **Early Stopping**: Early stopping wordt gebruikt met een `patience` van 20 epochs. Dit betekent dat de training stopt als het validatieverlies voor 20 opeenvolgende epochs niet verbetert, waarbij de beste modelgewichten worden hersteld.

Deze aanpassingen zijn gemaakt met het doel om de nauwkeurigheid van het model te verbeteren en overfitting te verminderen. Door het aantal neuronen te verhogen en dropout toe te passen, is het model beter in staat om de onderliggende patronen in de data te leren zonder te veel te leren van de ruis of specifieke details van de trainingsset.




In [ ]:
# Opslaan van het getrainde model (het tweede model) op de gespecificeerde locatie
model_save_path = '/content/drive/My Drive/Colab Notebooks/DeepLearning_2024/Opdracht 1/modelversies/fcnn_model_v2.h5'
model.save(model_save_path)




In [ ]:
from tensorflow.keras.models import load_model


model_load_path = '/content/drive/My Drive/Colab Notebooks/DeepLearning_2024/Opdracht 1/modelversies/fcnn_model_v2.h5'


model = load_model(model_load_path)
model.summary()

Met die code heb ik het eerste model opgeslagen in een nieuwe map genaamd modelversies

In [ ]:
def kaggle_csv(model, input_, suffix):
    """
    Deze functie maakt de voorspelling op de test dataset
    en vormt deze om tot een csv bestand om in te kunnen
    leveren op Kaggle.

    Parameters:
    ----------
    model : ML-model of DL-model
        De naam die is gegeven aan het ML-model of DL-model
        dat wordt gebruikt om te voorspellen

    suffix : str
        De 'tag' voor de naam van het csv
        bestand, zodat deze makkelijk te identificeren
        is na de submission.

    Returns:
    ----------
    None
        In plaats van een return maakt het een bestand
        aan in de map Kaggle Submissions. De namen zien
        er als volgt uit:

    """
    # Aanmaken van de voorspelling
    y_pred = model.predict(input_)

    # Aanmaken df met alleen House ID en Price
    test_predictions_df = pd.DataFrame(
        {'House ID': test_data['House ID'],
        'Price': np.ravel(y_pred)})

    # Aanmaken van tijd
    tijd = datetime.datetime.now().strftime("%m%d%H%M%S")

    # Aanmaken csv bestand met timestamp
    test_predictions_df.to_csv(
        f'Kaggle/voorspelling_{suffix}_{tijd}.csv',
        index=False)

    # Print voor conformatie
    print(f'voorspelling_{suffix}_{tijd}.csv has been saved!')

Nu zal de functie worden uitgevoerd om de voorspellingen te verkrijgen.

In [ ]:
# Uitvoeren van kaggle_csv functie
kaggle_csv(fcnn, sc_test_fe, 'FCNN-2')

- [ ] Bouw en train een dense (fully-connected) netwerk om de huizenprijzen te voorspellen op basis van de tabular data.


- [ ] Beschrijf welke lossfunctie het meest geschikt is voor dit probleem. Kies een geschikte optimizer en beschrijf kort hoe deze optimizer werkt.


- [ ] Plot je modelarchitectuur met behulp van een geschikte visualisatietool.
- [ ] Onderbouw je keuzes van het aantal convolutionele lagen, poollagen, convolutionele kernels per laag, het aantal fully-connected lagen en het aantal neuronen per laag.

## CNN from Scratch


- [ ] Bouw een convolutioneel neuraal netwerkmodel om de prijzen te voorspellen op basis van de afbeeldingen.


In [ ]:
from pathlib import Path

train_ds = tf.keras.utils.image_dataset_from_directory(
  "Train",
  seed=42,
  label_mode=None,
  image_size=(256, 256),
  batch_size=4)

# Extract file paths from the dataset
image_file_paths = train_ds.file_paths

# Extract only file ID without extension from file paths
image_ids = [int(Path(image_file_path).stem) for image_file_path in image_file_paths]

# Map each file path to its label using the DataFrame
# Ensure the uuids in the DataFrame exactly match those in `image_uuids`
labels = [train_data.loc[train_data['House ID'] == hid, 'Price'].values for hid in image_ids]

# Convert the list of labels into a TensorFlow dataset
labels_dataset = tf.data.Dataset.from_tensor_slices(labels).batch(4)

# Combine the original image dataset with the labels dataset
final_dataset = tf.data.Dataset.zip((train_ds, labels_dataset))

train_dataset = final_dataset.take(110)
val_dataset = final_dataset.skip(110)

# Define a function to normalize the images
def normalize_img(image, label):
    # Normalize the pixel values to the range [0, 1]
    return tf.cast(image, tf.float32) / 255.0, label

# Apply normalization to both training and validation datasets
train_dataset_normalized = train_dataset.map(normalize_img)
val_dataset_normalized = val_dataset.map(normalize_img)

In [ ]:
img_shape = (256, 256, 3)
batch_size = 50
epochs = 5000
optimizer = Adam(learning_rate=0.01)
loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsolutePercentageError()]

cnn_model = Sequential([
    keras.Input(shape=img_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, 'linear')
])

cnn_model.summary()

In [ ]:
cnn_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
    )

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

cnn_history = cnn_model.fit(
    train_dataset_normalized,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=val_dataset_normalized,
    callbacks=[early_stop]
)

- [ ] Plot je modelarchitectuur met behulp van een geschikte visualisatietool.
- [ ] Onderbouw je keuzes van het aantal lagen, het aantal neuronen per laag, de activeringsfuncties, de keuze van optimizer en het aantal trainingsepochs.


In [ ]:
#Plot modelarchitecture
keras.utils.plot_model(cnn_model, show_shapes=True)

## Keuzes van het model

### De lagen

1. **Inputlaag**: Dit is waar de gegevens binnenkomen. Het model verwacht afbeeldingen van 512 bij 512 pixels met 3 kleurkanalen.

2. **Convolutionele lagen (Conv2D)**: Deze lagen zijn als filters die door de afbeeldingen gaan om verschillende kenmerken te vinden, zoals randen, texturen en vormen. We gebruiken drie van deze lagen om op verschillende niveaus van detail te leren.

3. **Pooling-lagen (MaxPooling2D)**: Deze lagen verminderen de grootte van de gegevens door de resolutie te verkleinen, waardoor de complexiteit van het model afneemt en overfitting wordt voorkomen.

4. **Flatten-laag**: Hier worden de gegevens omgevormd van een 3D-structuur naar een eenvoudige lijst, zodat ze kunnen worden gebruikt in de volgende lagen.

5. **Dense (volledig verbonden) lagen**: Dit zijn lagen waar elk neuron verbonden is met elk neuron in de vorige laag. Ze leren complexere patronen en worden vaak gebruikt om de uiteindelijke voorspellingen te maken.

### Aantal neuronen per laag

### De actervingsfuncties

1. Voor Conv2D wordt de ReLU-activatiefunctie gebruikt, aangeduid met 'relu'. ReLU zet negatieve invoerwaarden om in nul en laat positieve waarden ongewijzigd, waardoor het model niet-lineaire kenmerken kan leren.

2. Voor de laatste dense laag wordt een lineaire activatiefunctie gebruikt. Voor prijsvoorspellingen op basis van afbeeldingen is het het beste om een lineaire activatiefunctie te gebruiken voor de laatste laag van het model. Dit past goed bij regressietaken waarbij het model een continue uitvoer moet voorspellen.

### Keuze van Adam optimizer

Goed voor veel variatie in de data: Adam past de leersnelheid aan voor elk onderdeel van het model, wat handig is als de gegevens veel variatie hebben.

Combineert voordelen: Het combineert de beste eigenschappen van twee andere optimizers (AdaGrad en RMSProp), waardoor het beter kan werken dan oudere methoden zoals SGD.

Minder instellingen nodig: Je hoeft niet zoveel handmatige instellingen te doen als bij andere methoden. Dit voorkomt veel tijd. (Brownlee, 2021)

Brownlee, J. (2021, January 12). Gentle introduction to the ADAM Optimization Algorithm for deep learning. MachineLearningMastery.com. https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/


### Keuze van aantal epochs

Wij hebben gekozen voor 5000 epochs om zo het mode met zekerheid te optimaliseren. We hebben van gebruikt gemaakt van early stopping om zo dagen aan tijd te besparen en overfitting te voorkomen.



- [ ] Gebruik een of meer van de volgende technieken in je model. Beschrijf en motiveer je keuze:
    - [ ] L1 regularisatie
    - [ ] L2 regularisatie
    - [ ] Dropout
    - [ ] Batchnorm

In [ ]:
# Use L2 and dropout in the model

# cnn_model2 = Sequential([
#     keras.Input(shape=img_shape),
#     Conv2D(filters=32, kernel_size=(3,3), activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
#     MaxPooling2D((2,2)),
#     Conv2D(filters=64, kernel_size=(3,3), activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
#     MaxPooling2D((2,2)),
#     Conv2D(filters=64, kernel_size=(3,3), activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dropout(0.2),
#     Dense(1, 'linear')
# ])

In [ ]:
# cnn_model2.compile(
#     optimizer=optimizer,
#     loss=loss,
#     metrics=metrics
#     )

In [ ]:
# cnn_history2 = cnn_model2.fit(
#     train_dataset_normalized,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_data=val_dataset_normalized,
#     callbacks=[early_stop]
# )

### Motivatie

Het combineren van L2-regularisatie en dropout kan een krachtige manier zijn om overfitting te verminderen en de algehele prestaties van het model te verbeteren. Door de gewichten te reguleren met L2-regularisatie en tegelijkertijd het model robuuster te maken met dropout, kan het model beter generaliseren naar nieuwe gegevens en nauwkeurigere voorspellingen doen. Dit kan vooral nuttig zijn bij het voorspellen van prijzen op basis van afbeeldingen, waar het belangrijk is om de relevante kenmerken van de afbeeldingen vast te leggen zonder te veel te vertrouwen op specifieke details die alleen relevant zijn voor de trainingsgegevens.


- [ ] Beschrijf welke vormen van data augmentation (indien van toepassing) geschikt zijn voor dit probleem. Implementeer dit in je datapijplijn.
- [ ] Compile je model met je gekozen lossfunctie en optimizer, en train deze op de afbeeldingen.

**Augmentation**  
- Padding: maakt extra pixels om de gehele image, dit wordt gebruikt om zo veel mogelijk informatie te behouden.  
- Stride: geeft aan met hoeveel pixels de kernel zich verschuift op de image. In het geval dat er niet genoeg pixels over zijn, bijvoorbeeld er is maar 1 row van pixels over na het shiften, dan gaat de hele row niet gebruikt worden.
- MaxPooling: pakt het hoogste waarde uit de kernel.

In [ ]:
# kernel_regularizer = keras.regularizers.l1(0.01)

cnn_model3 = Sequential([
    keras.Input(shape=img_shape),

    Conv2D(filters=32, kernel_size=(3,3), activation=None, padding='same'),
    BatchNormalization(),
    Activation('relu'),

    Conv2D(filters=64, kernel_size=(3,3), activation=None, padding='same'),
    BatchNormalization(),
    Activation('relu'),

    Conv2D(filters=64, kernel_size=(3,3), activation=None, padding='same'),
    BatchNormalization(),
    Activation('relu'),

    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, 'linear')
])

cnn_model3.summary()

cnn_model3.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
    )

cnn_history3 = cnn_model3.fit(
    train_dataset_normalized,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=val_dataset_normalized,
    callbacks=[early_stop]
)

# Transfer Learning

# Multimodaal model